# Setup

In [34]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.utils.tensorboard as tb

import ray
from ray import tune, train
from ray.tune.schedulers import ASHAScheduler
from ray.tune import CLIReporter
ray.init(ignore_reinit_error=True)
print(ray.cluster_resources())

import os
import pickle
import random

2024-11-27 13:45:09,911	INFO worker.py:1652 -- Calling ray.init() again after it has already been called.


{'CPU': 10.0, 'object_store_memory': 2147483648.0, 'node:127.0.0.1': 1.0, 'node:__internal_head__': 1.0, 'memory': 33902397031.0}


In [35]:
## SAVE FEATURE STATISTICS FROM PREPROCESSING ##
X_MEAN = 21.234056
Y_MEAN = -16.93773
Z_MEAN = 24.948428

HR_MEAN = 81.085869 # bpm
WEIGHT_MEAN = 73.272422 # kg
HEIGHT_MEAN = 172.332927 # cm
GENDER_MEAN = 0.721742 # 0:female, 1:male -- represents class imbalance
AGE_MEAN = 30.236566 # yrs

EE_MEAN = 3481.625380 # kcal/min

X_STD = 35.144448
Y_STD = 37.25645
Z_STD = 23.991521

HR_STD = 29.852576
WEIGHT_STD = 11.258137
HEIGHT_STD = 6.570272
GENDER_STD = 0.448141
AGE_STD = 5.199545

EE_STD = 2752.199323

# Model designs

## CNN feature generator

In [36]:
class CNNFeatureGenerator(nn.Module):
    def __init__(self, num_features, num_conv_layers=3):
        super(CNNFeatureGenerator, self).__init__()
        layers = []
        in_channels = num_features

        for i in range(num_conv_layers):
            out_channels = 16 * (2 ** i)  # Double channels with each layer
            layers.extend([
                nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                nn.ReLU()
            ])

            if i < num_conv_layers - 1:
                layers.append(nn.MaxPool1d(kernel_size=2, stride=2))

            in_channels = out_channels

        layers.append(nn.AdaptiveAvgPool1d(1))  # Final pooling to reduce to 1D
        self.feature_extractor = nn.Sequential(*layers)

    def forward(self, x):
        """
        Forward pass through the CNN feature extractor.

        Args:
            x: Input tensor of shape (batch_size, num_features, window_size).

        Returns:
            Tensor: Extracted features of shape (batch_size, out_channels).
        """
        x = self.feature_extractor(x)
        return x.view(x.size(0), -1)  # Flatten for FCN

## FCN Regressor

In [37]:
class FCNRegressor(nn.Module):
    def __init__(self, feature_size=64, num_fcn_layers=3, layer_width=128):
        super(FCNRegressor, self).__init__()
        layers = []
        for _ in range(num_fcn_layers - 1):
            layers.extend([
                nn.Linear(feature_size, layer_width),
                nn.ReLU(),
                nn.Dropout(0.5)
            ])
            feature_size = layer_width  # Update input size for next layer
            layer_width //= 2

        layers.append(nn.Linear(feature_size, 1))  # Final regression layer
        self.regressor = nn.Sequential(*layers)

    def forward(self, x):
        return self.regressor(x)

## CNN to FCN Model

In [38]:
class CNNFCNRegressor(nn.Module):
    def __init__(self, cnn_input_features=4, num_conv_layers=3, fcn_demo_features=4, num_fcn_layers=3, fcn_width=128):
        """
        Combines a CNN feature generator with an FCN regressor.

        Args:
            num_features: Number of input features (e.g., channels in the input tensor).
            num_conv_layers: Number of convolutional layers in the CNN.
            num_fcn_layers: Number of fully connected layers in the FCN.
            fcn_width: Width of the hidden layers in the FCN.
        """
        super(CNNFCNRegressor, self).__init__()

        self.cnn = CNNFeatureGenerator(num_features=cnn_input_features, num_conv_layers=num_conv_layers)
        conv_output_size = 16 * (2 ** (num_conv_layers - 1))  # Compute feature size based on CNN output channels
        self.fcn = FCNRegressor(feature_size=conv_output_size+fcn_demo_features, num_fcn_layers=num_fcn_layers, layer_width=fcn_width)

    def forward(self, temporal, demographic):
        """
        Forward pass through the combined model.

        Args:
            all_features: Tensor of shape (batch_size, num_features, window_size)

        Returns:
            Tensor: Predicted energy expenditure (batch_size, 1)
        """
        cnn_features = self.cnn(temporal)  # Extract features with CNN
        all_features = torch.cat([cnn_features, demographic], dim=1)
        output = self.fcn(all_features)  # Regress with FCN
        return output

## LSTM Module

In [39]:
class LSTMModule(nn.Module):
    def __init__(self, input_size=64, hidden_size=64, num_layers=2):
        super(LSTMModule, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.5
        )

    def forward(self, cnn_features):
        """
        Forward pass through the LSTM module.

        Args:
            cnn_features: Tensor of shape (batch_size, seq_len, input_size)

        Returns:
            Tensor: Output features from the LSTM (batch_size, hidden_size)
        """
        lstm_out, _ = self.lstm(cnn_features)  # lstm_out: (batch_size, seq_len, hidden_size)
        lstm_features = lstm_out[:, -1, :]  # Take the last timestep (batch_size, hidden_size)
        return lstm_features

## CNN to LSTM to FCN Model

In [40]:
class CNNLSTMFCNRegressor(nn.Module):
    def __init__(self, cnn_input_features=4, num_conv_layers=3, lstm_hidden_size=64, lstm_num_layers=2, fcn_demo_features=4, num_fcn_layers=3, fcn_width=128):
        super(CNNLSTMFCNRegressor, self).__init__()

        self.cnn = CNNFeatureGenerator(num_features=cnn_input_features, num_conv_layers=num_conv_layers)
        conv_output_size = 16 * (2 ** (num_conv_layers - 1))  # Compute feature size based on CNN output channels
        self.lstm = LSTMModule(input_size=conv_output_size, hidden_size=lstm_hidden_size, num_layers=lstm_num_layers)
        self.fcn = FCNRegressor(feature_size=lstm_hidden_size+fcn_demo_features, num_fcn_layers=num_fcn_layers, layer_width=fcn_width)  # Feature size matches LSTM hidden size

    def forward(self, temporal, demographic):
        """
        Forward pass through the CNN-LSTM-FCN regressor.

        Args:
            all_features: Tensor of shape (batch_size, num_features, window_size)

        Returns:
            Tensor: Predicted energy expenditure (batch_size, 1)
        """
        cnn_features = self.cnn(temporal)  # Extract features from CNN (batch_size, feature_size)
        lstm_input = cnn_features.unsqueeze(1)  # Add temporal dimension (batch_size, seq_len=1, feature_size)
        lstm_features = self.lstm(lstm_input)  # Pass through LSTM (batch_size, lstm_hidden_size)
        all_features = torch.cat([lstm_features, demographic], dim=1)
        output = self.fcn(all_features)  # Pass through FCN regressor (batch_size, 1)
        return output

# Load and prep data

## Load data

In [41]:
def display_windows(subject_windows_list):
        display(subject_windows_list[0])
        display(subject_windows_list[-1])

In [42]:
windows_path = 'data/WEEE Dataset/processed/windows.pkl'
with open(windows_path, 'rb') as f:
    windows_dict = pickle.load(f)

windows_dict.keys()

dict_keys(['P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17'])

In [43]:
display_windows(windows_dict['P01'])

,X,Y,Z,heart_rate,Weight,Height,Gender,Age,EE[kcal/min]
0,0.220972,0.454626,1.294273,0.455724,-0.237377,-0.963877,0,-0.237822,-0.422138
1,0.220972,0.454626,1.294273,0.455724,-0.237377,-0.963877,0,-0.237822,-0.422138
2,0.220972,0.454626,1.252592,0.455724,-0.237377,-0.963877,0,-0.237822,-0.422138
3,0.249426,0.454626,1.294273,0.455724,-0.237377,-0.963877,0,-0.237822,-0.422138
4,0.249426,0.454626,1.294273,0.455724,-0.237377,-0.963877,0,-0.237822,-0.422138
...,...,...,...,...,...,...,...,...,...
187,0.249426,0.454626,1.252592,0.449849,-0.237377,-0.963877,0,-0.237822,-0.384203
188,0.249426,0.454626,1.294273,0.449849,-0.237377,-0.963877,0,-0.237822,-0.384203
189,0.249426,0.454626,1.294273,0.449849,-0.237377,-0.963877,0,-0.237822,-0.384203
190,0.249426,0.454626,1.294273,0.449849,-0.237377,-0.963877,0,-0.237822,-0.384203


,X,Y,Z,heart_rate,Weight,Height,Gender,Age,EE[kcal/min]
0,-0.063568,-0.780061,0.794096,2.373159,-0.237377,-0.963877,0,-0.237822,0.846824
1,-0.205838,-0.806902,0.544008,2.373159,-0.237377,-0.963877,0,-0.237822,0.846824
2,-0.120476,-0.780061,0.877459,2.373159,-0.237377,-0.963877,0,-0.237822,0.846824
3,-0.262746,-0.860584,0.544008,2.373159,-0.237377,-0.963877,0,-0.237822,0.846824
4,-0.006660,-0.887425,0.335601,2.373159,-0.237377,-0.963877,0,-0.237822,0.846824
...,...,...,...,...,...,...,...,...,...
187,-0.376562,-0.806902,0.627371,2.003558,-0.237377,-0.963877,0,-0.237822,0.384886
188,0.021794,-0.833743,0.669052,2.003558,-0.237377,-0.963877,0,-0.237822,0.384886
189,-0.177384,-0.967948,0.168875,2.003558,-0.237377,-0.963877,0,-0.237822,0.384886
190,0.050248,-0.941107,0.377282,2.003558,-0.237377,-0.963877,0,-0.237822,0.384886


## Extract features

In [45]:
def extract_features(windows_dict, label_column='EE[kcal/min]', demographic_columns=['Weight', 'Height', 'Gender', 'Age']):
    """
    Prepares the temporal and demographic features and labels as PyTorch tensors 
    for leave-one-subject-out cross-validation.

    Args:
        windows_dict: Dictionary of preprocessed windows.
        label_column: The column to extract as labels.
        demographic_columns: List of columns representing demographic features.

    Returns:
        data_dict: A dictionary with subject-wise splits for temporal features, 
                   demographic features, and labels as PyTorch tensors.
    """
    data_dict = {}
    for subject_id, windows in windows_dict.items():
        temporal_features = []
        demographic_features = []
        labels = []
        
        for window in windows:
            # Separate demographic features and temporal features
            demographic_array = window[demographic_columns].iloc[0].values  # Single row for demographic data
            temporal_array = window.drop(columns=[label_column] + demographic_columns).values  # Remaining temporal features
            
            # Append features and labels
            temporal_features.append(temporal_array.T)  # Shape: (num_features, window_size)
            demographic_features.append(demographic_array)  # Shape: (num_demographic_features,)
            label_mean = window[label_column].mean()  # Average label for the window
            labels.append(label_mean)

        # Convert to PyTorch tensors and store
        data_dict[subject_id] = {
            'temporal_features': torch.tensor(np.stack(temporal_features), dtype=torch.float32),  # (num_windows, num_features, window_size)
            'demographic_features': torch.tensor(np.stack(demographic_features), dtype=torch.float32),  # (num_windows, num_demographic_features)
            'labels': torch.tensor(np.array(labels), dtype=torch.float32)  # (num_windows,)
        }
    return data_dict

In [46]:
data_dict = extract_features(windows_dict)
data_dict

{'P01': {'temporal_features': tensor([[[ 0.2210,  0.2210,  0.2210,  ...,  0.2494,  0.2494,  0.2210],
           [ 0.4546,  0.4546,  0.4546,  ...,  0.4546,  0.4546,  0.4546],
           [ 1.2943,  1.2943,  1.2526,  ...,  1.2943,  1.2943,  1.2943],
           [ 0.4557,  0.4557,  0.4557,  ...,  0.4498,  0.4498,  0.4498]],
  
          [[ 0.2494,  0.2494,  0.2494,  ...,  0.2494,  0.2494,  0.2210],
           [ 0.4546,  0.4546,  0.4546,  ...,  0.4546,  0.4546,  0.4546],
           [ 1.2943,  1.2943,  1.2943,  ...,  1.2943,  1.2943,  1.2943],
           [ 0.4766,  0.4766,  0.4766,  ...,  0.2828,  0.2828,  0.2828]],
  
          [[ 0.2494,  0.2494,  0.2494,  ...,  0.2494,  0.2494,  0.2494],
           [ 0.4546,  0.4546,  0.4546,  ...,  0.4546,  0.4546,  0.4546],
           [ 1.2943,  1.2943,  1.2943,  ...,  1.2943,  1.2943,  1.2943],
           [ 0.4784,  0.4784,  0.4784,  ...,  0.4593,  0.4593,  0.4593]],
  
          ...,
  
          [[-0.2627, -0.2058, -0.0351,  ..., -0.3766, -0.2627, -0.

In [47]:
data_dict['P01'].keys()

dict_keys(['temporal_features', 'demographic_features', 'labels'])

## LOO-CV Split

In [106]:
def leave_one_out_split(data_dict, exclude_keys=['P03', 'P04', 'P06', 'P11', 'P13', 'P14']):
    """
    Generator for leave-one-subject-out cross-validation splits with temporal and demographic features.

    Args:
        data_dict: Dictionary with subject-wise temporal features, demographic features, and labels as PyTorch tensors.

    Yields:
        train_data: Tuple of training temporal features, demographic features, and labels as PyTorch tensors.
        test_data: Tuple of testing temporal features, demographic features, and labels as PyTorch tensors.
    """
    data_dict = {k: v for k, v in data_dict.items() if k not in exclude_keys}
    subjects = list(data_dict.keys())

    for test_subject in subjects:
        train_temporal_features = []
        train_demographic_features = []
        train_labels = []
        
        for subject, data in data_dict.items():
            if subject == test_subject:
                test_temporal_features = data['temporal_features']
                test_demographic_features = data['demographic_features']
                test_labels = data['labels']
            else:
                train_temporal_features.append(data['temporal_features'])
                train_demographic_features.append(data['demographic_features'])
                train_labels.append(data['labels'])

        # Combine training data using PyTorch tensors
        train_temporal_features = torch.cat(train_temporal_features, dim=0)
        train_demographic_features = torch.cat(train_demographic_features, dim=0)
        train_labels = torch.cat(train_labels, dim=0)

        train_data = (train_temporal_features, train_demographic_features, train_labels)
        test_data = (test_temporal_features, test_demographic_features, test_labels)

        yield train_data, test_data, test_subject

In [98]:
sorted(data_dict.keys())

['P01',
 'P02',
 'P03',
 'P04',
 'P05',
 'P06',
 'P07',
 'P08',
 'P09',
 'P10',
 'P11',
 'P12',
 'P13',
 'P14',
 'P15',
 'P16',
 'P17']

In [99]:
loocv_splits = leave_one_out_split(data_dict)
loocv_splits

<generator object leave_one_out_split at 0x3513c33e0>

# Train Models

## Training functions

### Base

In [109]:
def train_model_base(config, train_data, test_data):
    """
    Training loop compatible with Ray Tune for hyperparameter optimization.

    Args:
        config: Dictionary of hyperparameters provided by Ray Tune.
        train_data: Tuple of training features and labels as PyTorch tensors. Includes data from all but one subject.
        - train_features: Training feature tensor (shape: [num_samples, num_features, window_size]).
        - train_labels: Training label tensor (shape: [num_samples]).
        test_data: Tuple of testing features and labels as PyTorch tensors. Includes data from the left-out subject.
        - test_features: Testing feature tensor (shape: [num_samples, num_features, window_size]).
        - test_labels: Testing label tensor (shape: [num_samples]).
    """
    # Device
    # device = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')
    device = torch.device(f'cuda:{ray.get_gpu_ids()[0]}' if torch.cuda.is_available() else 'cpu')
    if 'print' in config and config['print']:
        print(f'Using device: {device}')

    # Unpack config
    learning_rate = config['learning_rate']
    batch_size = config['batch_size']
    epochs = config['epochs']
    model_class = config['model']
    model_class_name = model_class.__name__

    CNN_INPUT_FEATURES = 4
    FCN_DEMO_FEATURES = 4
    num_conv_layers = config.get('num_conv_layers', 3)
    lstm_hidden_size = config.get('lstm_hidden_size', 64)
    lstm_num_layers = config.get('lstm_num_layers', 2)
    num_fcn_layers = config.get('num_fcn_layers', 3)
    fcn_width = config.get('fcn_width', 128)

    # Logging
    model_name = f'{model_class_name}_conv{num_conv_layers}_fcn{num_fcn_layers}_fcnw{fcn_width}_lr{learning_rate}_batch{batch_size}_epochs{epochs}'\
        if model_class_name == 'CNNFCNRegressor' else\
            f'{model_class_name}_conv{num_conv_layers}_fcn{num_fcn_layers}_fcnw{fcn_width}_lstm{lstm_num_layers}_lstmh{lstm_hidden_size}_lr{learning_rate}_batch{batch_size}_epochs{epochs}'
    if 'log_dir' in config:
        log_dir = config['log_dir']
        os.makedirs(log_dir, exist_ok=True)
        tb_logger = tb.SummaryWriter(os.path.join(log_dir, model_name))
    else:
        tb_logger = None

    # Model, loss, and optimizer
    model = model_class(
        cnn_input_features=CNN_INPUT_FEATURES,
        num_conv_layers=num_conv_layers,
        fcn_demo_features=FCN_DEMO_FEATURES,
        num_fcn_layers=num_fcn_layers,
        fcn_width=fcn_width
    ).to(device)\
    if model_class_name == 'CNNFCNRegressor' else model_class(
        cnn_input_features=CNN_INPUT_FEATURES,
        num_conv_layers=num_conv_layers,
        lstm_hidden_size=lstm_hidden_size,
        lstm_num_layers=lstm_num_layers,
        fcn_demo_features=FCN_DEMO_FEATURES,
        num_fcn_layers=num_fcn_layers,
        fcn_width=fcn_width
    ).to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoaders
    train_loader = DataLoader(TensorDataset(*train_data), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TensorDataset(*test_data), batch_size=batch_size, shuffle=False)

    # Early stopping
    patience = config.get('fold_early_stop', None) # using distinct per-fold early stopping patience, more strict compared to overall patience
    best_val_mse = float('inf')
    epochs_no_improve = 0

    global_step = 0
    for epoch in range(epochs):
        if 'print' in config and config['print']:
            print(f'Epoch: {epoch+1}/{epochs}')

        # Training
        model.train()
        train_mse = 0.0
        for temporal, demographic, labels in train_loader:
            temporal, demographic, labels = temporal.to(device), demographic.to(device), labels.to(device)

            # Forward pass
            outputs = model(temporal, demographic).squeeze(-1)
            # print(f'Outputs shape: {outputs.shape}')
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_mse += loss.item()

            global_step += 1

        train_mse /= len(train_loader)

        if tb_logger:
            tb_logger.add_scalar('train_mse', train_mse, global_step)

        # Validation
        model.eval()
        val_mse = 0.0
        r2_labels_list = []
        r2_outputs_list = []
        with torch.no_grad():
            for temporal, demographic, labels in test_loader:
                temporal, demographic, labels = temporal.to(device), demographic.to(device), labels.to(device)

                outputs = model(temporal, demographic).squeeze(-1)
                loss = criterion(outputs, labels)

                # MSE & R2 score
                val_mse += loss.item()
                # if labels.numel() > 0:
                r2_labels_list.append(labels.cpu().numpy())
                r2_outputs_list.append(outputs.cpu().detach().numpy())

        # Average MSE and R2 score
        val_mse /= len(test_loader)
        r2_labels = np.concatenate(r2_labels_list)
        r2_outputs = np.concatenate(r2_outputs_list)
        r2 = r2_score(r2_labels, r2_outputs)

        if tb_logger:
            tb_logger.add_scalar('val_mse', val_mse, global_step)

        if 'print' in config and config['print']:
            print(f"Epoch {epoch+1}/{epochs}, Train MSE: {train_mse:.4f}, Val MSE: {val_mse:.4f}")

        if (epoch + 1) % 5 == 0 and 'model_dir' in config:
            model_dir = config['model_dir']
            os.makedirs(model_dir, exist_ok=True)
            torch.save(model.state_dict(), os.path.join(model_dir, model_name+f'_epoch{epoch+1}.pt'))

        # Early stopping
        if patience:
            if val_mse < best_val_mse:
                best_val_mse = val_mse
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            if epochs_no_improve >= patience:
                if 'print' in config and config['print']:
                    print(f"Early stopping triggered at epoch {epoch+1}.")
                    print(f"Best Validation Loss: {best_val_mse} at epoch {epoch+1 - patience}")
                break

    # Final checkpoint
    if 'model_dir' in config:
            model_dir = config['model_dir']
            os.makedirs(model_dir, exist_ok=True)
            torch.save(model.state_dict(), os.path.join(model_dir, model_name+f'_epoch{epoch+1}_final.pth'))

    if 'tune' in config and config['tune']:
        tune_report = {
            'train_mse':train_mse,
            'val_mse':val_mse,
            'val_r2':r2,
            'model_name':model_name,
            'epoch':epoch+1
        }
        
        if 'loocv' in config and config['loocv']:
            return tune_report
        else:
            train.report(tune_report)

### LOO-CV

In [110]:
def train_model_loocv(config, data_dict):
    """
    Train the model with Leave-One-Out Cross-Validation (LOO-CV) and aggregate results for Ray Tune.

    Args:
        config: Dictionary of hyperparameters provided by Ray Tune.
        data_dict: Dictionary with subject-wise features and labels for LOO-CV.

    Reports:
        The average validation loss across all folds to Ray Tune.
    """
    train_mses = []
    val_mses = []
    val_r2s = []
    epochs = []

    config['loocv'] = True  # Flag to return results for each fold

    # Iterate over folds using LOO-CV
    for fold_idx, (train_data, test_data, test_subject_id) in enumerate(leave_one_out_split(data_dict)):
        fold_report = train_model_base(config, train_data, test_data)
        train_mses.append(fold_report['train_mse'])
        val_mses.append(fold_report['val_mse'])
        val_r2s.append(fold_report['val_r2'])
        epochs.append(fold_report['epoch'])

        # Report cumulative metrics after each fold
        cumulative_avg_val_mse = sum(val_mses) / len(val_mses)
        cumulative_avg_val_r2 = sum(val_r2s) / len(val_r2s)

        train.report(
            {
                'avg_val_mse': cumulative_avg_val_mse,
                'avg_val_r2': cumulative_avg_val_r2,
                'fold_idx': fold_idx + 1  # Track progress
            }
        )

    # Compute average validation loss across all folds
    avg_train_mse = sum(train_mses) / len(train_mses)
    avg_train_rmse = np.sqrt(avg_train_mse)

    avg_val_mse = sum(val_mses) / len(val_mses)
    avg_val_rmse = np.sqrt(avg_val_mse)
    avg_val_r2 = sum(val_r2s) / len(val_r2s)
    
    avg_epochs = sum(epochs) / len(epochs)

    # Report to Ray Tune
    train.report(
        {
            'avg_train_mse':avg_train_mse,
            'avg_train_rmse':avg_train_rmse,

            'avg_val_mse':avg_val_mse,
            'avg_val_rmse':avg_val_rmse,
            'avg_val_r2':avg_val_r2,

            'avg_epochs':avg_epochs,
            'model_name':fold_report['model_name']
        }
    )

## Tuning Wrapper

### Base

In [111]:
def tune_model_base(search_space, train_data, test_data):
    """
    Wrapper for hyperparameter tuning using Ray Tune.

    Args:
        search_space: Dictionary defining the hyperparameter search space.
        train_data: Tuple of training features and labels as PyTorch tensors.
        test_data: Tuple of testing features and labels as PyTorch tensors.
    """
    search_space['tune'] = True  # Ensure the training function reports to Ray Tune

    # CLI Reporter for better logging
    model_class_name = search_space['model'].__name__
    parameter_columns = [
        'learning_rate',
        'batch_size',

        'num_conv_layers',
        'num_fcn_layers',
        'fcn_width'
    ] if model_class_name == 'CNNFCNRegressor' else [
        'learning_rate',
        'batch_size',

        'num_conv_layers',
        'num_fcn_layers',
        'fcn_width',
        'lstm_hidden_size',
        'lstm_num_layers'
    ]

    reporter = CLIReporter(
        parameter_columns=parameter_columns,
        metric_columns=['train_mse', 'val_mse', 'val_r2', 'epoch', 'model_name']
    )

    # Scheduler (e.g., ASHA for early stopping and prioritization)
    scheduler = ASHAScheduler(
        metric='val_mse',
        mode='min',
        grace_period=search_space['early_stop_patience'],
        reduction_factor=2
    )

    # Run the hyperparameter search
    analysis = tune.run(
        tune.with_parameters(
            train_model_base,
            train_data=train_data,
            test_data=test_data
        ),
        config=search_space,
        scheduler=scheduler,
        progress_reporter=reporter
        # num_samples=search_space.get('num_samples', 10)
    )

    # Get the best trial
    best_trial = analysis.get_best_trial(metric='val_mse', mode='min')
    best_config = best_trial.config
    best_train_mse = best_trial.last_result['train_mse']
    best_val_mse = best_trial.last_result['val_mse']
    best_r2 = best_trial.last_result['val_r2']
    best_epoch = best_trial.last_result['epoch']

    print('Best Hyperparameters Found:', best_config)
    print('Best Train MSE:', best_train_mse)
    print('Best Validation MSE:', best_val_mse)
    print('Best R2 Score:', best_r2)
    print('Best Number of Epochs:', best_epoch)

    # If a model path is specified in the search space, print the model save location
    if 'model_dir' in search_space:
        best_model_path = f"{search_space['model_dir']}/{best_trial.last_result['model_name']}.pth"
        print("Best Model Path:", best_model_path)

### LOO-CV

In [112]:
def tune_model_loocv(search_space, data_dict):
    """
    Wrapper for hyperparameter tuning using Ray Tune with Leave-One-Out Cross-Validation.

    Args:
        search_space: Dictionary defining the hyperparameter search space.
        data_dict: Dictionary with subject-wise features and labels for LOO-CV.
    """
    search_space['tune'] = True  # Ensure the training function reports to Ray Tune

    # CLI Reporter for better logging
    model_class_name = search_space['model'].__name__
    parameter_columns = [
        'learning_rate',
        'batch_size',

        'num_conv_layers',
        'num_fcn_layers',
        'fcn_width'
    ] if model_class_name == 'CNNFCNRegressor' else [
        'learning_rate',
        'batch_size',

        'num_conv_layers',
        'num_fcn_layers',
        'fcn_width',
        'lstm_hidden_size',
        'lstm_num_layers'
    ]

    reporter = CLIReporter(
        parameter_columns=parameter_columns,
        metric_columns=['avg_train_mse', 'avg_train_rmse', 'avg_val_mse', 'avg_val_rmse', 'avg_val_r2', 'avg_epochs', 'model_name']
    )

    # Scheduler (e.g., ASHA for early stopping and prioritization)
    scheduler = ASHAScheduler(
        metric='avg_val_mse',
        mode='min',
        grace_period=search_space['early_stop_patience'],
        reduction_factor=2
    )

    # Run the hyperparameter search
    analysis = tune.run(
        tune.with_parameters(
            train_model_loocv,
            data_dict=data_dict
        ),
        config=search_space,
        scheduler=scheduler,
        progress_reporter=reporter,
        # resources_per_trial={"cpu": 1, "gpu": 1} if torch.cuda.is_available() else {"cpu": 1}
    )

    # Get the best trial
    best_trial = analysis.get_best_trial(metric='avg_val_mse', mode='min')
    best_config = best_trial.config
    best_avg_train_mse = best_trial.last_result['avg_train_mse']
    best_avg_train_rmse = best_trial.last_result['avg_train_rmse']
    best_avg_val_mse = best_trial.last_result['avg_val_mse']
    best_avg_val_rmse = best_trial.last_result['avg_val_rmse']
    best_avg_r2 = best_trial.last_result['avg_val_r2']
    best_avg_epochs = best_trial.last_result['avg_epochs']

    print('Best Hyperparameters Found:', best_config)
    print('Best Avg Train MSE:', best_avg_train_mse)
    print('Best Avg Train RMSE:', best_avg_train_rmse)
    print('Best Avg Validation MSE:', best_avg_val_mse)
    print('Best Avg Validation RMSE:', best_avg_val_rmse)
    print('Best Avg R2 Score:', best_avg_r2)
    print('Best Avg Number of Epochs:', best_avg_epochs)

    # If a model path is specified in the search space, print the model save location
    if 'model_dir' in search_space:
        best_model_path = f"{search_space['model_dir']}/{best_trial.last_result['model_name']}.pth"
        print('Best Model Path:', best_model_path)

## Test runs

### Tiny test set

In [57]:
def reduce_to_tiny_dataset(data_dict, num_windows=10):
    """
    Reduces the number of windows for each subject in the data_dict to a specified number.

    Args:
        data_dict: Dictionary with subject-wise splits for temporal, demographic features, and labels as PyTorch tensors.
        num_windows: Number of windows to retain per subject (default is 10).

    Returns:
        reduced_data_dict: A dictionary with reduced subject-wise splits.
    """
    reduced_data_dict = {}
    for subject_id, data in data_dict.items():
        temporal_features = data['temporal_features']
        demographic_features = data['demographic_features']
        labels = data['labels']

        num_available_windows = temporal_features.size(0)
        if num_available_windows > num_windows:
            # Randomly sample indices for reduction
            selected_indices = random.sample(range(num_available_windows), num_windows)
        else:
            # If there are fewer windows than num_windows, use all of them
            selected_indices = list(range(num_available_windows))

        # Select the sampled windows
        reduced_temporal_features = temporal_features[selected_indices]
        reduced_demographic_features = demographic_features[selected_indices]
        reduced_labels = labels[selected_indices]

        # Store the reduced data
        reduced_data_dict[subject_id] = {
            'temporal_features': reduced_temporal_features,
            'demographic_features': reduced_demographic_features,
            'labels': reduced_labels
        }
    return reduced_data_dict

In [60]:
reduced_data_dict = reduce_to_tiny_dataset(data_dict)
reduced_data_dict['P01'].keys()

dict_keys(['temporal_features', 'demographic_features', 'labels'])

In [61]:
reduced_data_dict['P01']['temporal_features'].shape

torch.Size([10, 4, 192])

### Test9 CNNFCN Separated Features -- Individual R2 Tests

In [ ]:
LOGS_CNNFCN_TEST9 = os.path.join(os.getcwd(), 'logs/CNNFCN/test9')
MODELS_CNNFCN_TEST9 = os.path.join(os.getcwd(), 'models/CNNFCN/test9')

# search_space_CNNFCN_test9 = {
#     'learning_rate': tune.grid_search([1e-3]), # 1e-4, 5e-3, 1e-3, 5e-2, 1e-2
#     'batch_size': tune.grid_search([16, 32]), # 16, 32, 64
#     'epochs': 100,

#     'model': CNNFCNRegressor,
#     'num_conv_layers':tune.grid_search([1, 2]), # 1, 2, 3
#     'num_fcn_layers':tune.grid_search([1, 2]), # 1, 2, 3
#     'fcn_width':tune.grid_search([32]), # , 64, 128

#     'log_dir': LOGS_CNNFCN_TEST9,
#     'model_dir': MODELS_CNNFCN_TEST9,

#     # 'num_samples': 4,
#     'early_stop_patience': 20,
#     'fold_early_stop': 20,

#     'print': False,
#     'tune': True
#     # 'loocv': True
# }

In [79]:
# tune_model_loocv(search_space_CNNFCN_test9, reduced_data_dict)

In [85]:
search_space_CNNFCN_test9 = {
    'learning_rate': tune.grid_search([1e-3]), # 1e-4, 5e-3, 1e-3, 5e-2, 1e-2
    'batch_size': tune.grid_search([16]), # 16, 32, 64
    'epochs': 100,

    'model': CNNFCNRegressor,
    'num_conv_layers':tune.grid_search([1]), # 1, 2, 3
    'num_fcn_layers':tune.grid_search([1]), # 1, 2, 3
    'fcn_width':tune.grid_search([32]), # , 64, 128

    'log_dir': LOGS_CNNFCN_TEST9,
    'model_dir': MODELS_CNNFCN_TEST9,

    # 'num_samples': 4,
    'early_stop_patience': 20,
    'fold_early_stop': 20,

    'print': False,
    'tune': True
    # 'loocv': False
}

In [ ]:
# for fold_idx, (train_data, test_data, test_subject_id) in enumerate(leave_one_out_split(data_dict)):
#     print(f'\nFold {test_subject_id}\n')
#     tune_model_base(search_space_CNNFCN_test9, train_data, test_data)

2024-11-27 14:38:21,921	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949



Fold P01

== Status ==
Current time: 2024-11-27 14:38:22 (running for 00:00:00.11)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical resource usage: 1.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-11-27_13-22-53_167611_40329/artifacts/2024-11-27_14-38-21/train_model_base_2024-11-27_14-38-21/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2024-11-27 14:38:27 (running for 00:00:05.15)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical resource usage: 1.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-11-27_13-22-53_167611_40329/artifacts/2024-11-27_14-38-21/train_model_base_2024-11-27_14-38-21/driver_artifacts
Number of trials: 1/1 (1 RUNNING)


== Status ==
Current time: 2024-11-27 14:38:32 (running for 00:00:10.22)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None

Trial name,epoch,model_name,train_mse,val_mse,val_r2
train_model_base_8aee4_00000,77,CNNFCNRegressor_conv1_fcn1_fcnw32_lr0.001_batch16_epochs100,0.250209,0.40522,0.381179


2024-11-27 14:39:03,897	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/tyler/ray_results/train_model_base_2024-11-27_14-38-21' in 0.0055s.
2024-11-27 14:39:03,899	INFO tune.py:1041 -- Total run time: 41.98 seconds (41.96 seconds for the tuning loop).
2024-11-27 14:39:03,912	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2024-11-27 14:39:03 (running for 00:00:41.97)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical resource usage: 1.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-11-27_13-22-53_167611_40329/artifacts/2024-11-27_14-38-21/train_model_base_2024-11-27_14-38-21/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)
+------------------------------+------------+-----------------+-----------------+--------------+-------------------+------------------+-------------+-------------+-----------+----------+---------+----------------------+
| Trial name                   | status     | loc             |   learning_rate |   batch_size |   num_conv_layers |   num_fcn_layers |   fcn_width |   train_mse |   val_mse |   val_r2 |   epoch | model_name           |
|------------------------------+------------+-----------------+-----------------+--------------+-------------------+------------------+-------------+---

...

2024-11-27 14:43:45,105	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/tyler/ray_results/train_model_base_2024-11-27_14-43-32' in 0.0054s.
2024-11-27 14:43:45,107	INFO tune.py:1041 -- Total run time: 12.45 seconds (12.44 seconds for the tuning loop).
2024-11-27 14:43:45,120	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2024-11-27 14:43:45 (running for 00:00:12.44)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical resource usage: 1.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-11-27_13-22-53_167611_40329/artifacts/2024-11-27_14-43-32/train_model_base_2024-11-27_14-43-32/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)
+------------------------------+------------+-----------------+-----------------+--------------+-------------------+------------------+-------------+-------------+-----------+----------+---------+----------------------+
| Trial name                   | status     | loc             |   learning_rate |   batch_size |   num_conv_layers |   num_fcn_layers |   fcn_width |   train_mse |   val_mse |   val_r2 |   epoch | model_name           |
|------------------------------+------------+-----------------+-----------------+--------------+-------------------+------------------+-------------+---

Trial name,epoch,model_name,train_mse,val_mse,val_r2
train_model_base_4b925_00000,40,CNNFCNRegressor_conv1_fcn1_fcnw32_lr0.001_batch16_epochs100,0.270849,0.699504,0.645071


2024-11-27 14:44:05,667	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/tyler/ray_results/train_model_base_2024-11-27_14-43-45' in 0.0054s.
2024-11-27 14:44:05,669	INFO tune.py:1041 -- Total run time: 20.55 seconds (20.54 seconds for the tuning loop).


== Status ==
Current time: 2024-11-27 14:44:05 (running for 00:00:20.54)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical resource usage: 1.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-11-27_13-22-53_167611_40329/artifacts/2024-11-27_14-43-45/train_model_base_2024-11-27_14-43-45/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)
+------------------------------+------------+-----------------+-----------------+--------------+-------------------+------------------+-------------+-------------+-----------+----------+---------+----------------------+
| Trial name                   | status     | loc             |   learning_rate |   batch_size |   num_conv_layers |   num_fcn_layers |   fcn_width |   train_mse |   val_mse |   val_r2 |   epoch | model_name           |
|------------------------------+------------+-----------------+-----------------+--------------+-------------------+------------------+-------------+---

### Test9 CNNLSTMFCN Separated Features -- Individual R2 Tests

In [ ]:
LOGS_CNNLSTMFCN_TEST7 = os.path.join(os.getcwd(), 'logs/CNNLSTMFCN/test7')
MODELS_CNNLSTMFCN_TEST7 = os.path.join(os.getcwd(), 'models/CNNLSTMFCN/test7')

# search_space_CNNLSTMFCN_test7 = {
#     'learning_rate': tune.grid_search([1e-3]), # 1e-4, 5e-3, 1e-3, 5e-2, 1e-2
#     'batch_size': tune.grid_search([16, 32]), # 16, 32, 64
#     'epochs': 50,

#     'model': CNNLSTMFCNRegressor,
#     'num_conv_layers':tune.grid_search([1, 2]), # 1, 2, 3
#     'num_fcn_layers':tune.grid_search([1, 2]), # 1, 2, 3
#     'fcn_width':tune.grid_search([32]), # , 64, 128

#     'lstm_hidden_size':tune.grid_search([32]), # , 64, 128
#     'lstm_num_layers':tune.grid_search([1]), # , 2, 3

#     'log_dir': LOGS_CNNLSTMFCN_TEST7,
#     'model_dir': MODELS_CNNLSTMFCN_TEST7,

#     # 'num_samples': 4,
#     'early_stop_patience': 5,

#     'print': False,
#     'tune': True,
#     'loocv': True
# }

In [ ]:
# tune_model_loocv(search_space_CNNLSTMFCN_test7, data_dict)

In [92]:
LOGS_CNNLSTMFCN_TEST9 = os.path.join(os.getcwd(), 'logs/CNNLSTMFCN/test9')
MODELS_CNNLSTMFCN_TEST9 = os.path.join(os.getcwd(), 'models/CNNLSTMFCN/test9')

search_space_CNNLSTMFCN_test9 = {
    'learning_rate': tune.grid_search([1e-3]), # 1e-4, 5e-3, 1e-3, 5e-2, 1e-2
    'batch_size': tune.grid_search([16]), # 16, 32, 64
    'epochs': 100,

    'model': CNNLSTMFCNRegressor,
    'num_conv_layers':tune.grid_search([1]), # 1, 2, 3
    'num_fcn_layers':tune.grid_search([1]), # 1, 2, 3
    'fcn_width':tune.grid_search([32]), # , 64, 128

    'lstm_hidden_size':tune.grid_search([32]), # , 64, 128
    'lstm_num_layers':tune.grid_search([1]), # , 2, 3

    'log_dir': LOGS_CNNFCN_TEST9,
    'model_dir': MODELS_CNNFCN_TEST9,

    # 'num_samples': 4,
    'early_stop_patience': 20,
    'fold_early_stop': 20,

    'print': False,
    'tune': True
    # 'loocv': False
}

In [ ]:
# for fold_idx, (train_data, test_data, test_subject_id) in enumerate(leave_one_out_split(data_dict)):
#     print(f'\nFold {test_subject_id}\n')
#     tune_model_base(search_space_CNNLSTMFCN_test9, train_data, test_data)

2024-11-27 14:48:39,101	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949



Fold P01

== Status ==
Current time: 2024-11-27 14:48:39 (running for 00:00:00.11)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical resource usage: 1.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-11-27_13-22-53_167611_40329/artifacts/2024-11-27_14-48-39/train_model_base_2024-11-27_14-48-39/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2024-11-27 14:48:44 (running for 00:00:05.14)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical resource usage: 1.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-11-27_13-22-53_167611_40329/artifacts/2024-11-27_14-48-39/train_model_base_2024-11-27_14-48-39/driver_artifacts
Number of trials: 1/1 (1 RUNNING)


== Status ==
Current time: 2024-11-27 14:48:49 (running for 00:00:10.22)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None

Trial name,epoch,model_name,train_mse,val_mse,val_r2
train_model_base_facc7_00000,30,CNNLSTMFCNRegressor_conv1_fcn1_fcnw32_lstm1_lstmh32_lr0.001_batch16_epochs100,0.200611,0.411939,0.384018


2024-11-27 14:49:01,282	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/tyler/ray_results/train_model_base_2024-11-27_14-48-39' in 0.0065s.
2024-11-27 14:49:01,284	INFO tune.py:1041 -- Total run time: 22.18 seconds (22.17 seconds for the tuning loop).
2024-11-27 14:49:01,297	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2024-11-27 14:49:01 (running for 00:00:22.17)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical resource usage: 1.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-11-27_13-22-53_167611_40329/artifacts/2024-11-27_14-48-39/train_model_base_2024-11-27_14-48-39/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)
+------------------------------+------------+-----------------+-----------------+--------------+-------------------+------------------+-------------+--------------------+-------------------+-------------+-----------+----------+---------+----------------------+
| Trial name                   | status     | loc             |   learning_rate |   batch_size |   num_conv_layers |   num_fcn_layers |   fcn_width |   lstm_hidden_size |   lstm_num_layers |   train_mse |   val_mse |   val_r2 |   epoch | model_name           |
|------------------------------+------------+-----------------+-------

...

2024-11-27 14:53:51,713	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/tyler/ray_results/train_model_base_2024-11-27_14-53-27' in 0.0066s.
2024-11-27 14:53:51,715	INFO tune.py:1041 -- Total run time: 24.08 seconds (24.07 seconds for the tuning loop).
2024-11-27 14:53:51,728	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2024-11-27 14:53:51 (running for 00:00:24.08)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical resource usage: 1.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-11-27_13-22-53_167611_40329/artifacts/2024-11-27_14-53-27/train_model_base_2024-11-27_14-53-27/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)
+------------------------------+------------+-----------------+-----------------+--------------+-------------------+------------------+-------------+--------------------+-------------------+-------------+-----------+----------+---------+----------------------+
| Trial name                   | status     | loc             |   learning_rate |   batch_size |   num_conv_layers |   num_fcn_layers |   fcn_width |   lstm_hidden_size |   lstm_num_layers |   train_mse |   val_mse |   val_r2 |   epoch | model_name           |
|------------------------------+------------+-----------------+-------

Trial name,epoch,model_name,train_mse,val_mse,val_r2
train_model_base_b5236_00000,29,CNNLSTMFCNRegressor_conv1_fcn1_fcnw32_lstm1_lstmh32_lr0.001_batch16_epochs100,0.185065,0.668863,0.664099


2024-11-27 14:54:10,357	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/tyler/ray_results/train_model_base_2024-11-27_14-53-51' in 0.0062s.
2024-11-27 14:54:10,358	INFO tune.py:1041 -- Total run time: 18.63 seconds (18.61 seconds for the tuning loop).


== Status ==
Current time: 2024-11-27 14:54:10 (running for 00:00:18.62)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical resource usage: 1.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-11-27_13-22-53_167611_40329/artifacts/2024-11-27_14-53-51/train_model_base_2024-11-27_14-53-51/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)
+------------------------------+------------+-----------------+-----------------+--------------+-------------------+------------------+-------------+--------------------+-------------------+-------------+-----------+----------+---------+----------------------+
| Trial name                   | status     | loc             |   learning_rate |   batch_size |   num_conv_layers |   num_fcn_layers |   fcn_width |   lstm_hidden_size |   lstm_num_layers |   train_mse |   val_mse |   val_r2 |   epoch | model_name           |
|------------------------------+------------+-----------------+-------

### Test10 CNNFCN Removed Subjects - Full dataset

In [95]:
LOGS_CNNFCN_TEST10 = os.path.join(os.getcwd(), 'logs/CNNFCN/test10')
MODELS_CNNFCN_TEST10 = os.path.join(os.getcwd(), 'models/CNNFCN/test10')

search_space_CNNFCN_test10 = {
    'learning_rate': tune.grid_search([1e-3]), # 1e-4, 5e-3, 1e-3, 5e-2, 1e-2
    'batch_size': tune.grid_search([16, 32]), # 16, 32, 64
    'epochs': 100,

    'model': CNNFCNRegressor,
    'num_conv_layers':tune.grid_search([1, 2]), # 1, 2, 3
    'num_fcn_layers':tune.grid_search([1, 2]), # 1, 2, 3
    'fcn_width':tune.grid_search([32]), # , 64, 128

    'log_dir': LOGS_CNNFCN_TEST10,
    'model_dir': MODELS_CNNFCN_TEST10,

    # 'num_samples': 4,
    'early_stop_patience': 20,
    'fold_early_stop': 20,

    'print': False,
    'tune': True
    # 'loocv': True
}

In [105]:
tune_model_loocv(search_space_CNNFCN_test10, data_dict)

2024-11-27 15:04:55,397	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2024-11-27 15:04:55 (running for 00:00:00.19)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical resource usage: 8.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-11-27_13-22-53_167611_40329/artifacts/2024-11-27_15-04-55/train_model_loocv_2024-11-27_15-04-55/driver_artifacts
Number of trials: 8/8 (8 PENDING)


== Status ==
Current time: 2024-11-27 15:05:00 (running for 00:00:05.29)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical resource usage: 8.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-11-27_13-22-53_167611_40329/artifacts/2024-11-27_15-04-55/train_model_loocv_2024-11-27_15-04-55/driver_artifacts
Number of trials: 8/8 (8 RUNNING)


== Status ==
Current time: 2024-11-27 15:05:05 (running for 00:00:10.31)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical 

Trial name,avg_epochs,avg_train_mse,avg_train_rmse,avg_val_mse,avg_val_r2,avg_val_rmse,model_name
train_model_loocv_40b76_00000,45.3636,0.30572,0.55292,0.587869,0.416228,0.766726,CNNFCNRegressor_conv1_fcn1_fcnw32_lr0.001_batch16_epochs100
train_model_loocv_40b76_00001,55.8182,0.312524,0.559038,0.622054,0.392963,0.788704,CNNFCNRegressor_conv1_fcn1_fcnw32_lr0.001_batch32_epochs100
train_model_loocv_40b76_00002,28.8182,0.196152,0.44289,0.709438,0.328636,0.842281,CNNFCNRegressor_conv2_fcn1_fcnw32_lr0.001_batch16_epochs100
train_model_loocv_40b76_00003,34.6364,0.198493,0.445526,0.702346,0.249205,0.838061,CNNFCNRegressor_conv2_fcn1_fcnw32_lr0.001_batch32_epochs100
train_model_loocv_40b76_00004,45.8182,0.307829,0.554823,0.833336,0.0753904,0.912872,CNNFCNRegressor_conv1_fcn2_fcnw32_lr0.001_batch16_epochs100
train_model_loocv_40b76_00005,46.4545,0.320043,0.565724,0.724375,0.192316,0.851102,CNNFCNRegressor_conv1_fcn2_fcnw32_lr0.001_batch32_epochs100
train_model_loocv_40b76_00006,29.1818,0.247723,0.497718,0.673102,0.374704,0.820428,CNNFCNRegressor_conv2_fcn2_fcnw32_lr0.001_batch16_epochs100
train_model_loocv_40b76_00007,31.4545,0.252396,0.50239,0.662093,0.380233,0.813691,CNNFCNRegressor_conv2_fcn2_fcnw32_lr0.001_batch32_epochs100


== Status ==
Current time: 2024-11-27 15:05:10 (running for 00:00:15.33)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical resource usage: 8.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-11-27_13-22-53_167611_40329/artifacts/2024-11-27_15-04-55/train_model_loocv_2024-11-27_15-04-55/driver_artifacts
Number of trials: 8/8 (8 RUNNING)


== Status ==
Current time: 2024-11-27 15:05:15 (running for 00:00:20.37)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical resource usage: 8.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-11-27_13-22-53_167611_40329/artifacts/2024-11-27_15-04-55/train_model_loocv_2024-11-27_15-04-55/driver_artifacts
Number of trials: 8/8 (8 RUNNING)


== Status ==
Current time: 2024-11-27 15:05:20 (running for 00:00:25.44)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical 

2024-11-27 15:11:57,076	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/tyler/ray_results/train_model_loocv_2024-11-27_15-04-55' in 0.0117s.
2024-11-27 15:11:57,078	INFO tune.py:1041 -- Total run time: 421.68 seconds (421.66 seconds for the tuning loop).


== Status ==
Current time: 2024-11-27 15:11:57 (running for 00:07:01.67)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical resource usage: 1.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-11-27_13-22-53_167611_40329/artifacts/2024-11-27_15-04-55/train_model_loocv_2024-11-27_15-04-55/driver_artifacts
Number of trials: 8/8 (8 TERMINATED)
+-------------------------------+------------+----------------+-----------------+--------------+-------------------+------------------+-------------+-----------------+------------------+---------------+----------------+--------------+--------------+----------------------+
| Trial name                    | status     | loc            |   learning_rate |   batch_size |   num_conv_layers |   num_fcn_layers |   fcn_width |   avg_train_mse |   avg_train_rmse |   avg_val_mse |   avg_val_rmse |   avg_val_r2 |   avg_epochs | model_name           |
|-------------------------------+------------

### Test10 CNNLSTMFCN Removed Subjects - Full dataset

In [114]:
LOGS_CNNLSTMFCN_TEST10 = os.path.join(os.getcwd(), 'logs/CNNLSTMFCN/test10')
MODELS_CNNLSTMFCN_TEST10 = os.path.join(os.getcwd(), 'models/CNNLSTMFCN/test10')

search_space_CNNLSTMFCN_test10 = {
    'learning_rate': tune.grid_search([1e-3]), # 1e-4, 5e-3, 1e-3, 5e-2, 1e-2
    'batch_size': tune.grid_search([16, 32]), # 16, 32, 64
    'epochs': 100,

    'model': CNNLSTMFCNRegressor,
    'num_conv_layers':tune.grid_search([1, 2]), # 1, 2, 3
    'num_fcn_layers':tune.grid_search([1, 2]), # 1, 2, 3
    'fcn_width':tune.grid_search([32]), # , 64, 128

    'lstm_hidden_size':tune.grid_search([32]), # , 64, 128
    'lstm_num_layers':tune.grid_search([1, 2]), # , 2, 3

    'log_dir': LOGS_CNNLSTMFCN_TEST10,
    'model_dir': MODELS_CNNLSTMFCN_TEST10,

    # 'num_samples': 4,
    'early_stop_patience': 20,
    'fold_early_stop': 20,

    'print': False,
    'tune': True
}

In [115]:
tune_model_loocv(search_space_CNNLSTMFCN_test10, data_dict)

2024-11-27 16:00:57,566	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2024-11-27 16:00:57 (running for 00:00:00.24)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical resource usage: 0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-11-27_13-22-53_167611_40329/artifacts/2024-11-27_16-00-57/train_model_loocv_2024-11-27_16-00-57/driver_artifacts
Number of trials: 16/16 (16 PENDING)


== Status ==
Current time: 2024-11-27 16:01:02 (running for 00:00:05.26)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical resource usage: 10.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-11-27_13-22-53_167611_40329/artifacts/2024-11-27_16-00-57/train_model_loocv_2024-11-27_16-00-57/driver_artifacts
Number of trials: 16/16 (6 PENDING, 10 RUNNING)


== Status ==
Current time: 2024-11-27 16:01:07 (running for 00:00:10.33)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.00

Trial name,avg_epochs,avg_train_mse,avg_train_rmse,avg_val_mse,avg_val_r2,avg_val_rmse,model_name
train_model_loocv_14b9f_00000,37.3636,0.213135,0.461666,0.679938,0.286983,0.824584,CNNLSTMFCNRegressor_conv1_fcn1_fcnw32_lstm1_lstmh32_lr0.001_batch16_epochs100
train_model_loocv_14b9f_00001,46.2727,0.226343,0.475755,0.615191,0.407882,0.784341,CNNLSTMFCNRegressor_conv1_fcn1_fcnw32_lstm1_lstmh32_lr0.001_batch32_epochs100
train_model_loocv_14b9f_00002,44.2727,0.231203,0.480835,0.70329,0.239825,0.838624,CNNLSTMFCNRegressor_conv1_fcn1_fcnw32_lstm2_lstmh32_lr0.001_batch16_epochs100
train_model_loocv_14b9f_00003,47.2727,0.241012,0.490929,0.712143,0.243927,0.843886,CNNLSTMFCNRegressor_conv1_fcn1_fcnw32_lstm2_lstmh32_lr0.001_batch32_epochs100
train_model_loocv_14b9f_00004,27.8182,0.187389,0.432884,0.647261,0.329036,0.804525,CNNLSTMFCNRegressor_conv2_fcn1_fcnw32_lstm1_lstmh32_lr0.001_batch16_epochs100
train_model_loocv_14b9f_00005,33.5455,0.193118,0.439452,0.68355,0.316629,0.826771,CNNLSTMFCNRegressor_conv2_fcn1_fcnw32_lstm1_lstmh32_lr0.001_batch32_epochs100
train_model_loocv_14b9f_00006,29,0.201578,0.448974,0.650935,0.313097,0.806805,CNNLSTMFCNRegressor_conv2_fcn1_fcnw32_lstm2_lstmh32_lr0.001_batch16_epochs100
train_model_loocv_14b9f_00007,31.4545,0.204755,0.452498,0.718036,0.248724,0.84737,CNNLSTMFCNRegressor_conv2_fcn1_fcnw32_lstm2_lstmh32_lr0.001_batch32_epochs100
train_model_loocv_14b9f_00008,35.7273,0.270111,0.519722,0.62933,0.387436,0.793303,CNNLSTMFCNRegressor_conv1_fcn2_fcnw32_lstm1_lstmh32_lr0.001_batch16_epochs100
train_model_loocv_14b9f_00009,46.2727,0.267637,0.517337,0.652752,0.37711,0.807931,CNNLSTMFCNRegressor_conv1_fcn2_fcnw32_lstm1_lstmh32_lr0.001_batch32_epochs100


== Status ==
Current time: 2024-11-27 16:01:18 (running for 00:00:20.45)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical resource usage: 10.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-11-27_13-22-53_167611_40329/artifacts/2024-11-27_16-00-57/train_model_loocv_2024-11-27_16-00-57/driver_artifacts
Number of trials: 16/16 (6 PENDING, 10 RUNNING)


== Status ==
Current time: 2024-11-27 16:01:23 (running for 00:00:25.49)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical resource usage: 10.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-11-27_13-22-53_167611_40329/artifacts/2024-11-27_16-00-57/train_model_loocv_2024-11-27_16-00-57/driver_artifacts
Number of trials: 16/16 (6 PENDING, 10 RUNNING)


== Status ==
Current time: 2024-11-27 16:01:28 (running for 00:00:30.56)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: Non

2024-11-27 16:16:42,530	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/tyler/ray_results/train_model_loocv_2024-11-27_16-00-57' in 0.0145s.
2024-11-27 16:16:42,534	INFO tune.py:1041 -- Total run time: 944.97 seconds (944.94 seconds for the tuning loop).


== Status ==
Current time: 2024-11-27 16:16:42 (running for 00:15:44.95)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80.000: None | Iter 40.000: None | Iter 20.000: None
Logical resource usage: 1.0/10 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-11-27_13-22-53_167611_40329/artifacts/2024-11-27_16-00-57/train_model_loocv_2024-11-27_16-00-57/driver_artifacts
Number of trials: 16/16 (16 TERMINATED)
+-------------------------------+------------+-----------------+-----------------+--------------+-------------------+------------------+-------------+--------------------+-------------------+-----------------+------------------+---------------+----------------+--------------+--------------+----------------------+
| Trial name                    | status     | loc             |   learning_rate |   batch_size |   num_conv_layers |   num_fcn_layers |   fcn_width |   lstm_hidden_size |   lstm_num_layers |   avg_train_mse |   avg_train_rmse |   avg_val_mse |   avg_val_rmse |   avg_val_r

# Experiment Runs

### Run1 CNNFCN

In [ ]:
LOGS_CNNFCN_RUN1 = os.path.join(os.getcwd(), 'logs/CNNFCN/run1')
MODELS_CNNFCN_RUN1 = os.path.join(os.getcwd(), 'models/CNNFCN/run1')

search_space_CNNFCN_run1 = {
    'learning_rate': tune.grid_search([1e-4, 5e-3, 1e-3, 5e-2, 1e-2]), # 1e-4, 5e-3, 1e-3, 5e-2, 1e-2
    'batch_size': tune.grid_search([16, 32, 64]), # 16, 32, 64
    'epochs': 100,

    'model': CNNFCNRegressor,
    'num_conv_layers':tune.grid_search([1, 2, 3]), # 1, 2, 3
    'num_fcn_layers':tune.grid_search([1, 2, 3]), # 1, 2, 3
    'fcn_width':tune.grid_search([32, 64]), # 32, 64, 128

    'log_dir': LOGS_CNNFCN_RUN1,
    'model_dir': MODELS_CNNFCN_RUN1,

    # 'num_samples': 4,
    'early_stop_patience': 20,
    'fold_early_stop': 20,

    'print': False,
    'tune': True
}

In [ ]:
# tune_model_loocv(search_space_CNNFCN_run1, data_dict)

### Run1 CNNLSTMFCN

In [ ]:
LOGS_CNNLSTMFCN_RUN1 = os.path.join(os.getcwd(), 'logs/CNNLSTMFCN/run1')
MODELS_CNNLSTMFCN_RUN1 = os.path.join(os.getcwd(), 'models/CNNLSTMFCN/run1')

search_space_CNNLSTMFCN_run1 = {
    'learning_rate': tune.grid_search([1e-4, 5e-3, 1e-3, 5e-2, 1e-2]), # 1e-4, 5e-3, 1e-3, 5e-2, 1e-2
    'batch_size': tune.grid_search([16, 32, 64]), # 16, 32, 64
    'epochs': 100,

    'model': CNNLSTMFCNRegressor,
    'num_conv_layers':tune.grid_search([1, 2, 3]), # 1, 2, 3
    'num_fcn_layers':tune.grid_search([1, 2, 3]), # 1, 2, 3
    'fcn_width':tune.grid_search([32, 64]), # 32, 64, 128

    'lstm_hidden_size':tune.grid_search([32, 64]), # 32, 64, 128
    'lstm_num_layers':tune.grid_search([1, 2, 3]), # 1, 2, 3

    'log_dir': LOGS_CNNLSTMFCN_RUN1,
    'model_dir': MODELS_CNNLSTMFCN_RUN1,

    # 'num_samples': 4,
    'early_stop_patience': 20,
    'fold_early_stop': 20,

    'print': False,
    'tune': True
}

In [ ]:
# tune_model_loocv(search_space_CNNLSTMFCN_run1, data_dict)